# `DDL`: Using `xarray` in `DiscreteDistribution`


First we import relevant libraries and tools, including the new `DiscreteDistributionLabeled` class.


In [ ]:
import numpy as np
from HARK.distribution import (
    MeanOneLogNormal,
    DiscreteDistributionLabeled,
    calc_expectation,
    combine_indep_dstns,
)

We create a distribution of shocks to income from continuous distributions.


In [ ]:
PermShkDstn = MeanOneLogNormal().discretize(200)
TranShkDstn = MeanOneLogNormal().discretize(200)
IncShkDstn = combine_indep_dstns(PermShkDstn, TranShkDstn)

Taking the components of `IncShkDstn`, we can now create a `DiscreteDistributionLabeled` object. As a demonstration of additional features, we can add a name attribute to the `DDL` object, as well as named dimensions and coordinates.


In [ ]:
x_dist = DiscreteDistributionLabeled.from_unlabeled(
    IncShkDstn,
    name="Distribution of Shocks to Income",
    var_names=["perm_shk", "tran_shk"],
    var_attrs=[
        {
            "name": "Permanent Shocks to Income",
            "limit": {"type": "Lognormal", "mean": -0.5, "variance": 1.0},
        },
        {
            "name": "Transitory Shocks to Income",
            "limit": {"type": "Lognormal", "mean": -0.5, "variance": 1.0},
        },
    ],
)

The underlying object and metadata is stored in a `xarray.Dataset` object which can be accessed using the `.dataset` attribute.


In [ ]:
x_dist.dataset

### Using functions with labels to take expresive expectations.


Taking the expectation of a `DDL` object is straightforward using the own `expected()` method.


In [ ]:
x_dist.expected()

As in the `DiscreteDistribution`, we can provide a function and arguments to the `expected()` method.


In [ ]:
aGrid = np.linspace(0, 20, 100)
R = 1.03

The main difference is that the `expected()` method of `DDL` objects can take a function that uses the labels of the `xarray.DataArray` object. This allows for clearer and more expresive mathematical functions and transition equations. Surprisingly, using a function with labels does not add much overhead to the function evaluation.


In [ ]:
%%timeit
x_dist.expected(
    lambda dist, a, R: R * a / dist["perm_shk"] + dist["tran_shk"],
    aGrid,
    R,
)

Compared to the old method of `calc_expectation` which takes a `DiscreteDistribution` object as input, the new method which takes a `DiscreteDistributionLabeled` object is significantly faster.

In [ ]:
%%timeit
calc_expectation(IncShkDstn, lambda dist, a, R: R * a / dist[0] + dist[1], aGrid, R)

We can also use `HARK.distribution.expected`.


In [ ]:
from HARK.distribution import expected

In [ ]:
expected(
    func=lambda dist, a, R: R * a / dist["perm_shk"] + dist["tran_shk"],
    dist=x_dist,
    args=(aGrid, R),
)

Additionally, we can use xarrays as inputs via keyword arguments.

In [ ]:
from xarray import DataArray

aNrm = DataArray(aGrid, name="aNrm", dims=("aNrm"))

In [ ]:
def mNrm_next(dist, R, a=None):
    variables = {}
    variables["mNrm_next"] = R * a / dist["perm_shk"] + dist["tran_shk"]
    return variables

In [ ]:
%%timeit
expected(
    func=mNrm_next,
    dist=x_dist,
    args=R,
    a=aNrm,
)

Taking the expectation with xarray inputs and labeled equations is still significantly faster than the old method.